In [3]:
#All the general import
import arxiv
import pandas as pd
import numpy as np
import pickle
import operator
from datetime import datetime
from dateutil.parser import parse
import calendar

### Download Data from Arxiv

In [4]:
req_count = 100000
step = 1000
start = 0
items = []

In [5]:
if (int(req_count/step) > 1):
    for i in range(int(req_count/step)):
        temp = []
        while (len(temp) == 0):
            temp = arxiv.query('learning', start = i*step, max_results = step)
        items.extend(temp)
        print('Retrieved: ', (i+1)*step, len(items), len(temp))

Retrieved:  1000 1000 1000
Retrieved:  2000 2000 1000
Retrieved:  3000 3000 1000
Retrieved:  4000 4000 1000
Retrieved:  5000 5000 1000
Retrieved:  6000 6000 1000
Retrieved:  7000 7000 1000
Retrieved:  8000 8000 1000
Retrieved:  9000 9000 1000
Retrieved:  10000 10000 1000
Retrieved:  11000 11000 1000
Retrieved:  12000 12000 1000
Retrieved:  13000 13000 1000
Retrieved:  14000 14000 1000
Retrieved:  15000 15000 1000
Retrieved:  16000 16000 1000
Retrieved:  17000 17000 1000
Retrieved:  18000 18000 1000
Retrieved:  19000 19000 1000
Retrieved:  20000 20000 1000
Retrieved:  21000 21000 1000
Retrieved:  22000 22000 1000
Retrieved:  23000 23000 1000
Retrieved:  24000 24000 1000
Retrieved:  25000 25000 1000
Retrieved:  26000 26000 1000
Retrieved:  27000 27000 1000
Retrieved:  28000 28000 1000
Retrieved:  29000 29000 1000
Retrieved:  30000 30000 1000
Retrieved:  31000 31000 1000
Retrieved:  32000 32000 1000
Retrieved:  33000 33000 1000
Retrieved:  34000 34000 1000
Retrieved:  35000 35000 1000
Ret

KeyboardInterrupt: 

In [ ]:
with open('./pickles/data_get/arxiv.pickle', 'wb') as handle:
    pickle.dump(items, handle)

### Load for data processing

In [29]:
with open('./pickles/data_get/arxiv.pickle', 'rb') as handle:
    data = pickle.load(handle)

#### Print unique tags

In [30]:
tags = {}
for item in data:
    for tag in item['tags']:
        if tag['term'] in tags:
            tags[tag['term']] += 1
        else:
            tags[tag['term']] = 0
tags = sorted(tags.items(), key = operator.itemgetter(1))
tags.reverse()
tags_only = set([i[0] for i in tags[0:50]])

In [31]:
avoid_tags = set(('physics.bio-p','quant-ph', 'hep-ph', 'q-bio.NC',\
                  'astro-ph.IM', 'astro-ph.CO', 'astro-ph.IM',\
                  'physics.ed-ph', 'physics.acc-ph', 'physics.soc-ph',\
                'physics.ins-det', 'gr-qc'))
##OR##
accept = ['cs.', 'math.', 'stat.']
reject = ['physics','hep'] #Random second word to avoid character looping
tags_clean = set([])
for tag in tags_only:
    contains = False
    for acc in accept:
        if acc in tag: contains = True
    for rej in reject:
        if rej in tag:
            contains = False
    if contains: tags_clean.add(tag)

In [32]:
print(tags_clean)

{'cs.PL', 'cs.CV', 'stat.CO', 'cs.NE', 'cs.SD', 'cs.IR', 'cs.CR', 'cs.SE', 'cs.RO', 'cs.DS', 'stat.TH', 'cs.IT', 'cs.NI', 'cs.GT', 'cs.SY', 'cs.DC', 'math.IT', 'stat.ME', 'cs.LG', 'cs.CL', 'cs.SI', 'cs.CY', 'cs.DB', 'cs.AI', 'stat.AP', 'cs.NA', 'cs.CC', 'math.ST', 'cs.LO', 'stat.ML', 'cs.CE', 'math.OC', 'math.NA', 'cs.HC', 'math.PR'}


In [33]:
def remove_bad_tags(data, clean_tags):
    to_remove = []
    for i in range(len(data)):
        should_remove = True
        for tag in data[i]['tags']:
            if tag['term'] in clean_tags:
                should_remove = False
        if should_remove: to_remove.append(i)
    for i in sorted(to_remove, reverse=True): #Neat way to remove things if to_remove is in order from small to large
        del data[i]
    return data

In [34]:
def clean_entry(data, indx):
    id_cur = data[indx]['id']
    id_cur = id_cur[id_cur.rfind('/')+1:]
    date_str = data[indx]['published']
    date = int(parse(date_str).timestamp())
    tags = [tag['term'] for tag in data[indx]['tags']]
    out_tup = [data[indx]['title'], data[indx]['summary'],  date, data[indx]['authors'], tags, id_cur]
    return out_tup

In [35]:
data = remove_bad_tags(data, tags_clean)

In [36]:
clean_data = [0]*len(data)
for i in range(len(data)):
    clean_data[i] = clean_entry(data, i)
data = pd.DataFrame(clean_data, columns = ['title', 'summary', 'date', 'authors', 'tags', 'id'])

#### To CSV (for backup)

In [37]:
data.head()

,title,summary,date,authors,tags,id
0,Minimax deviation strategies for machine learn...,The article is devoted to the problem of small...,1500196508,"[Michail Schlesinger, Evgeniy Vodolazskiy]",[cs.LG],1707.04849v1
1,mlbench: How Good Are Machine Learning Clouds ...,We conduct an empirical study of machine learn...,1501365558,"[Hantian Zhang, Luyuan Zeng, Wentao Wu, Ce Zhang]","[cs.DC, cs.LG, stat.ML]",1707.09562v2
2,Introduction to Machine Learning: Class Notes ...,Introduction to Machine learning covering Stat...,1240486857,[Amnon Shashua],[cs.LG],0904.3664v1
3,AutoCompete: A Framework for Machine Learning ...,"In this paper, we propose AutoCompete, a highl...",1436368059,"[Abhishek Thakur, Artus Krohn-Grimberghe]","[stat.ML, cs.LG]",1507.02188v1
4,Joint Training of Deep Boltzmann Machines,We introduce a new method for training deep Bo...,1355277567,"[Ian Goodfellow, Aaron Courville, Yoshua Bengio]","[stat.ML, cs.LG]",1212.2686v1


In [38]:
data.to_csv('./pickles/data_pd.csv')